<a href="https://colab.research.google.com/github/yagniksram007/Neural_Networks_Lab/blob/main/NN_Lab_Programs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Program 3

In [1]:
# Build a deep feed forward ANN by implementing the backpropagation algorithm an test the same using approopriate dataset. Use the hidden layers >=4

import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

# Helper Functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1 - x)

def softmax(x):
    exps = np.exp(x - np.max(x))
    return exps / np.sum(exps, axis=1, keepdims=True)

def cross_entropy_loss(y_true, y_pred):
    n_samples = y_true.shape[0]
    logp = - np.log(y_pred[range(n_samples), y_true.argmax(axis=1)])
    loss = np.sum(logp) / n_samples
    return loss

def cross_entropy_derivative(y_true, y_pred):
    return y_pred - y_true

# Initialize Weights
def initialize_weights(input_size, hidden_sizes, output_size):
    weights = []
    biases = []

    # Input to first hidden layer
    weights.append(np.random.randn(input_size, hidden_sizes[0]))
    biases.append(np.zeros((1, hidden_sizes[0])))

    # Hidden layers
    for i in range(1, len(hidden_sizes)):
        weights.append(np.random.randn(hidden_sizes[i-1], hidden_sizes[i]))
        biases.append(np.zeros((1, hidden_sizes[i])))

    # Hidden to output layer
    weights.append(np.random.randn(hidden_sizes[-1], output_size))
    biases.append(np.zeros((1, output_size)))

    return weights, biases

# Feedforward
def feedforward(X, weights, biases):
    layers = [X]
    for i in range(len(weights) - 1):
        Z = np.dot(layers[-1], weights[i]) + biases[i]
        A = sigmoid(Z)
        layers.append(A)

    Z_out = np.dot(layers[-1], weights[-1]) + biases[-1]
    A_out = softmax(Z_out)
    layers.append(A_out)

    return layers

# Backpropagation
def backpropagation(y_true, weights, biases, layers, learning_rate):
    y_pred = layers[-1]
    deltas = [cross_entropy_derivative(y_true, y_pred)]

    # Backpropagation through hidden layers
    for i in reversed(range(len(weights) - 1)):
        delta = np.dot(deltas[-1], weights[i + 1].T) * sigmoid_derivative(layers[i + 1])
        deltas.append(delta)

    deltas.reverse()

    # Update weights and biases
    for i in range(len(weights)):
        weights[i] -= learning_rate * np.dot(layers[i].T, deltas[i])
        biases[i] -= learning_rate * np.sum(deltas[i], axis=0, keepdims=True)

# Train Function
def train(X, y, input_size, hidden_sizes, output_size, epochs, learning_rate):
    weights, biases = initialize_weights(input_size, hidden_sizes, output_size)

    for epoch in range(epochs):
        layers = feedforward(X, weights, biases)
        loss = cross_entropy_loss(y, layers[-1])
        backpropagation(y, weights, biases, layers, learning_rate)

        if epoch % 100 == 0:
            print(f"Epoch {epoch}/{epochs} - Loss: {loss}")

    return weights, biases

# Predict Function
def predict(X, weights, biases):
    layers = feedforward(X, weights, biases)
    return np.argmax(layers[-1], axis=1)

# Load and Prepare Dataset (MNIST)
def load_mnist():
    mnist = fetch_openml('mnist_784')
    X = mnist.data
    y = mnist.target.astype(int)

    # Normalize
    X = X / 255.0

    # One hot encode labels
    enc = OneHotEncoder()
    y = enc.fit_transform(y.reshape(-1, 1)).toarray()

    return train_test_split(X, y, test_size=0.2, random_state=42)

# Main Program
if __name__ == '__main__':
    # Load data
    X_train, X_test, y_train, y_test = load_mnist()

    # Parameters
    input_size = 784  # 28x28 pixels
    hidden_sizes = [128, 64, 64, 32]  # 4 hidden layers
    output_size = 10  # 10 classes (digits 0-9)
    epochs = 1000
    learning_rate = 0.01

    # Train model
    weights, biases = train(X_train, y_train, input_size, hidden_sizes, output_size, epochs, learning_rate)

    # Test model
    y_pred = predict(X_test, weights, biases)
    y_test_labels = np.argmax(y_test, axis=1)

    accuracy = np.mean(y_pred == y_test_labels)
    print(f"Test Accuracy: {accuracy * 100:.2f}%")


AttributeError: 'Series' object has no attribute 'reshape'

Program 4

In [2]:
# Use the concept of regularization and dropout while designing the CNN model. Use the Fashion MNIST datasets. Record the Training accuracy and Test accuracy corresponding to the following architectures:
# a. Base Model
# b. Model with L1 Regularization
# c. Model with L2 Regularization
# d. Model with Dropout

import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import fashion_mnist
import matplotlib.pyplot as plt

# Step 1: Load the Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train = x_train.reshape(x_train.shape[0], 28, 28, 1).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28, 1).astype('float32') / 255

# Convert labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Function to plot accuracy
def plot_accuracy(history, title):
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(title)
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.show()

# Step 2: Build and compile the models

# Base Model
def build_base_model():
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Model with L1 Regularization
def build_l1_model():
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1),
                      kernel_regularizer=regularizers.l1(0.001)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu',
                      kernel_regularizer=regularizers.l1(0.001)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l1(0.001)),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Model with L2 Regularization
def build_l2_model():
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1),
                      kernel_regularizer=regularizers.l2(0.001)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu',
                      kernel_regularizer=regularizers.l2(0.001)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Model with Dropout
def build_dropout_model():
    model = models.Sequential([
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),  # Dropout layer to prevent overfitting
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Step 3: Train and Evaluate each Model
def train_and_evaluate(model, title):
    history = model.fit(x_train, y_train,
                        validation_data=(x_test, y_test),
                        epochs=10, batch_size=128)
    test_loss, test_acc = model.evaluate(x_test, y_test)
    print(f"{title} Test Accuracy: {test_acc * 100:.2f}%")
    plot_accuracy(history, title)

# Base Model
base_model = build_base_model()
train_and_evaluate(base_model, "Base Model")

# Model with L1 Regularization
l1_model = build_l1_model()
train_and_evaluate(l1_model, "L1 Regularization Model")

# Model with L2 Regularization
l2_model = build_l2_model()
train_and_evaluate(l2_model, "L2 Regularization Model")

# Model with Dropout
dropout_model = build_dropout_model()
train_and_evaluate(dropout_model, "Dropout Model")


29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10


KeyboardInterrupt: 